In [1]:
pip install sounddevice


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webrtcvad


  Using cached webrtcvad-2.0.10.tar.gz (66 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp312-cp312-win_amd64.whl size=19103 sha256=ceb7c58b22eb318aecb5c243d27138d5fcfea3d5e8fe76bc03c95b4a5a62017b
  Stored in directory: c:\users\leo_h\appdata\local\pip\cache\wheels\1e\d3\95\680fa3b16848f1a58d2edaed34c496224c89a9bc63e17b3614
Successfully built webrtcvad
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U openai-whisper

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install webrtcvad

Note: you may need to restart the kernel to use updated packages.


In [15]:
import sounddevice as sd
import numpy as np
import webrtcvad
import whisper
import collections
import struct
import re

SAMPLE_RATE = 16000
FRAME_DURATION_MS = 30  # ms
FRAME_SIZE = int(SAMPLE_RATE * FRAME_DURATION_MS / 1000)
CHANNELS = 1
VAD_AGGRESSIVENESS = 2  # 0–3: higher = more aggressive
MAX_SILENCE_SECONDS = 1.0

# === Load Whisper ===
model = whisper.load_model("base")

# === Setup VAD ===
vad = webrtcvad.Vad(VAD_AGGRESSIVENESS) 



In [16]:
GOODBYE_PATTERNS = [
    r"\b(bye|goodbye|see you|exit|quit|farewell)\b",
    r"talk to you later",
    r"that's all",
    r"that's it"
]

def float32_to_int16(audio):
    return (audio * 32767).astype(np.int16)

def is_speech(frame_bytes):
    return vad.is_speech(frame_bytes, SAMPLE_RATE)

def is_goodbye(text):
    for pattern in GOODBYE_PATTERNS:
        if re.search(pattern, text, re.IGNORECASE):
            return True
    return False

def record_until_silence():
    print("Listening...")

    buffer = []
    silence_buffer = collections.deque(maxlen=int(MAX_SILENCE_SECONDS * 1000 / FRAME_DURATION_MS))
    stream = sd.InputStream(samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='float32', blocksize=FRAME_SIZE)

    with stream:
        while True:
            audio_chunk, _ = stream.read(FRAME_SIZE)
            audio_chunk = audio_chunk.flatten()
            audio_int16 = float32_to_int16(audio_chunk)
            frame_bytes = struct.pack(f"{len(audio_int16)}h", *audio_int16)

            if is_speech(frame_bytes):
                buffer.append(audio_chunk)
                silence_buffer.clear()
            else:
                silence_buffer.append(audio_chunk)
                if len(silence_buffer) == silence_buffer.maxlen and len(buffer) > 0:
                    print("Silence detected, stopping...")
                    break

    full_audio = np.concatenate(buffer)
    return full_audio

# === Conversation Loop ===
def start_conversation():
    print("Start speaking. Say 'goodbye' to end the conversation.")
    while True:
        audio = record_until_silence()
        if len(audio) == 0:
            continue  # skip empty audio

        print("Transcribing...")
        # Whisper expects audio in float32, 16kHz mono
        result = model.transcribe(audio, fp16=False)
        text = result.get("text", "").strip()

        print(f"You said: {text}")
        if is_goodbye(text):
            print("Goodbye detected. Ending conversation.")
            break



In [17]:
start_conversation()

Start speaking. Say 'goodbye' to end the conversation.
Listening...
Silence detected, stopping...
Transcribing...
You said: I hope you enjoyed. Oh, I named this serpage. It was nice talking to you. That's it.
Goodbye detected. Ending conversation.
